# Neural Networks

In [52]:
import numpy as np
import pandas as pd
from sklearn.neural_network import MLPRegressor, MLPClassifier
from sklearn.model_selection import cross_validate

## AND perceptron with MLPClassifier

Create data for AND

In [24]:
x1 = np.array([0, 0, 1, 1])
x2 = np.array([0, 1, 0, 1])
y = np.array([0, 0, 0, 1])
X = np.column_stack((x1, x2))
print("X:\n", X, "\ny=", y)

X:
 [[0 0]
 [0 1]
 [1 0]
 [1 1]] 
y= [0 0 0 1]


In [28]:
m = MLPClassifier(hidden_layer_sizes=tuple(), 
                  max_iter=100000
                 ).fit(X, y)
yhat = m.predict(X)
print("Predicted probabilities:", m.predict_proba(X)[:,1])
print("Predicted outcomes:", yhat)

Predicted probabilities: [0.0839505  0.25929512 0.2241219  0.52458076]
Predicted outcomes: [0 0 0 1]


It does not work well--apparently MLPClassifier is not made for this kind of task.  But let's look at the fitted parameters:

In [29]:
print("weights:\n", m.coefs_, "intercepts", m.intercepts_)

weights:
 [array([[1.14803779],
       [1.34020767]])] intercepts [array([-2.38984309])]


As you can see, it comes up with a rather different solution than what we did manually.

## XOR Perceptron example

Lets see if we can get XOR perceptron to work in the same way.
$X$ will be the same, but we have to change $y$

In [19]:
x1 = np.array([0, 0, 1, 1])
x2 = np.array([0, 1, 0, 1])
y = np.array([0, 1, 1, 0])
X = np.column_stack((x1, x2))
print("X:\n", X, "\ny=", y)

X:
 [[0 0]
 [0 1]
 [1 0]
 [1 1]] 
y= [0 1 1 0]


In [45]:
m = MLPClassifier(hidden_layer_sizes = (2,),
                  max_iter=15000,
                  alpha=0
                 )
m.fit(X, y)
print("Estimation results:")
print("layers:", m.n_layers_)
print("coefficients:\n", m.coefs_)
# ordered by layers
print("intercepts:\n", m.intercepts_)
# ordered by layers
print("score:", m.score(X, y))
print("Predicted probability:", m.predict_proba(X)[:,1])
print("X1, X2, Y, predicted Y")
print(np.column_stack((X, y, m.predict(X))))



Estimation results:
layers: 3
coefficients:
 [array([[0.44461442, 1.76849546],
       [0.65336925, 1.57641954]]), array([[-1.01039601],
       [ 2.66568124]])]
intercepts:
 [array([-1.12334534, -1.57638046]), array([-2.61902145])]
score: 1.0
Predicted probability: [0.06792422 0.06793082 0.10842759 0.89044825]
X1, X2, Y, predicted Y
[[0 0 0 0]
 [0 1 0 0]
 [1 0 0 0]
 [1 1 1 1]]


It works, usually. 

# Predict republican/democrat

Use the same data we used in the lab.  Predict who is republican/democrat

In [49]:
vote = pd.read_csv("../data/house-votes-84.csv.bz2",
                   header=None,
                   names = ["party"] + ["x" + str(i) for i in range(16)])
# now replace all '?'-s with n-s:
vote[vote.iloc[:, range(1,17)] == "?"] = "n"
print(vote.head())
print("Data size:", vote.shape)

        party x0 x1 x2 x3 x4 x5 x6 x7 x8 x9 x10 x11 x12 x13 x14 x15
0  republican  n  y  n  y  y  y  n  n  n  y   n   y   y   y   n   y
1  republican  n  y  n  y  y  y  n  n  n  n   n   y   y   y   n   n
2    democrat  n  y  y  n  y  y  n  n  n  n   y   n   y   y   n   n
3    democrat  n  y  y  n  n  y  n  n  n  n   y   n   y   n   n   y
4    democrat  y  y  y  n  y  y  n  n  n  n   y   n   y   y   y   y
Data size: (435, 17)


In [56]:
X = np.where(vote.iloc[:,range(1,17)] == "y", 1, 0)
# we need a numeric matrix here
y = vote["party"]
m = MLPClassifier(hidden_layer_sizes = (5,),
                  max_iter=10000
                   )
m.fit(X, y)
yhat = m.predict(X)
print("accuracy on training data:", np.mean(yhat == y))
print(pd.crosstab(yhat, y))

accuracy on training data: 0.967816091954023
party       democrat  republican
row_0                           
democrat         260           7
republican         7         161


Well, but that was training data.  What about validation data?  Let's cross-validate!

In [57]:
cv = cross_validate(m, X, y, scoring="accuracy", cv=10)
print("CV accuracy:", np.mean(cv["test_score"]))

CV accuracy: 0.9468111346018322


It is a bit worse than a training data, but not too much worse.